# **Tool Binding**

**Tool binding** is the step where you **register tools with lanhuahe model(LLM)** so that: 
1. the LLM knowns that what tools are available
2. It knows **What each tool does** (via description)
3. It known **what input format to use** (via schema)

In [1]:
from dotenv import load_dotenv
import requests

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

In [2]:
@tool

def multiply(a:int, b:int)->int:
    """Given 2 numbers a and b this tool return their product"""
    return a * b

In [3]:
multiply.invoke({"a" : 3, "b" : 4})

12

In [4]:
llm = ChatOpenAI()

In [5]:
llm_with_tools = llm.bind_tools([multiply])

In [6]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x115f1dfa0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x115f407a0>, root_client=<openai.OpenAI object at 0x115dd9c10>, root_async_client=<openai.AsyncOpenAI object at 0x115f1e000>, model_kwargs={}, openai_api_key=SecretStr('**********'))

# **Tool Calling**

**Tool calling** is the process where the **LLM(Language model)** decides, during a conversation or task, that it needs to **use a specific tool**(function) -- and generates a structured output with:
- the **name of the tool**
- and the **arguments** to call it with 

In [7]:
llm_with_tools.invoke("Hi how are you?")

AIMessage(content="Hello! I'm here and ready to help. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 59, 'total_tokens': 78, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQwVi34XMOA5azmlj5DIYRuZ2ayMw', 'finish_reason': 'stop', 'logprobs': None}, id='run-67bc4028-d692-45c1-9e45-4b1d3b7bb8ff-0', usage_metadata={'input_tokens': 59, 'output_tokens': 19, 'total_tokens': 78, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
answer = llm_with_tools.invoke("Can you multiple 5 with 10?")

In [12]:
answer.tool_calls[0]['args']

{'a': 5, 'b': 10}

# **Tool Execution**

**Tool execution** is the step where the **actual Python function(tool)** is run using the input arguments that the **LLM suggested during tool calling**. 

In [13]:
multiply.invoke(answer.tool_calls[0]['args'])

50

In [14]:
multiply.invoke(answer.tool_calls[0])

ToolMessage(content='50', name='multiply', tool_call_id='call_BZMJZ2UuWLOahnMHYKrX9bCn')

In [42]:
query = HumanMessage("can you multiply 50 with 10?")

In [43]:
messages = [query]

In [44]:
answer = llm_with_tools.invoke(messages)

In [45]:
messages.append(answer)

In [46]:
messages

[HumanMessage(content='can you multiply 50 with 10?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KAsHBku41BvH58aU9WuN8Duw', 'function': {'arguments': '{"a":50,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 63, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQwjMV0S13kigZSi4kMdPaVgXwF6n', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-794401dd-402c-4093-b47f-3a0113f8b9e1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 50, 'b': 10}, 'id': 'call_KAsHBku41BvH58aU9WuN8Duw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 18, 

In [47]:
tool_result = multiply.invoke(answer.tool_calls[0])

In [48]:
messages.append(tool_result)

In [49]:
messages

[HumanMessage(content='can you multiply 50 with 10?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KAsHBku41BvH58aU9WuN8Duw', 'function': {'arguments': '{"a":50,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 63, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQwjMV0S13kigZSi4kMdPaVgXwF6n', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-794401dd-402c-4093-b47f-3a0113f8b9e1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 50, 'b': 10}, 'id': 'call_KAsHBku41BvH58aU9WuN8Duw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 18, 

In [50]:
llm_with_tools.invoke(messages).content

'The product of 50 and 10 is 500.'

# **Tool Create**

In [89]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [90]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [91]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [92]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745712001,
 'time_last_update_utc': 'Sun, 27 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745798401,
 'time_next_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4301}

In [93]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [94]:
llm = ChatOpenAI()

In [95]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [96]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [97]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [98]:
ai_message = llm_with_tools.invoke(messages)

In [99]:
messages.append(ai_message)

In [100]:
ai_message.tool_calls[0]

{'name': 'get_conversion_factor',
 'args': {'base_currency': 'INR', 'target_currency': 'USD'},
 'id': 'call_fyJom9wzP65wDYcqBTA1Z324',
 'type': 'tool_call'}

In [101]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)


In [102]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_fyJom9wzP65wDYcqBTA1Z324', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_ukxFvGutY87sweuPlhFcVzEJ', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 123, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BQwuTzuPrAh3mjf3HTPHMmFXJoiu7', 'finish_reason': 'too

In [103]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01171. \n\nBased on this conversion factor, 10 INR is equal to 0.1171 USD.'